# simulation

In [1]:
from utils import (load_problem_data,
                   load_solution)
from evaluation_nofail import evaluation_function, get_known, get_evaluation
import evaluation_nofail
import evaluation as evaluation_fail
import pandas as pd
import numpy as np

demand, datacenters, servers, selling_prices = load_problem_data(path='./data_test')

In [2]:



def parse_action_string(action_string):
    server_generation, actions, action_params = action_string.split("|")
    actions = actions.split("-")
    action_params = action_params.split("-")
    action_comb = [parse_action_comb_param(server_generation, action, action_param) for action, action_param in zip(actions, action_params)]
    action_comb = [action for action in action_comb if action is not None]
    return action_comb
    
def parse_action_comb_param(server_generation, action, action_param):
    if action == "buy":
        datacenter_id, average_U = action_param.split(",")
        average_U = float(average_U)
        return {"action": action, "datacenter_id": datacenter_id, "average_U": average_U, "server_generation": server_generation}
    elif action == "dismiss":
        dismiss_age = int(action_param)
        return {"action": action, "dismiss_age": dismiss_age, "server_generation": server_generation}
    elif action == "move":
        datacenter_id, average_U, move_age = action_param.split(",")
        average_U = float(average_U)
        move_age = int(move_age)
        return {"action": action, "datacenter_id": datacenter_id, "average_U": average_U, "move_age": move_age,}
    return None

action_comb = parse_action_string("GPU.S3|buy-dismiss|DC3,0.5-84")
action_comb


[{'action': 'buy',
  'datacenter_id': 'DC3',
  'average_U': 0.5,
  'server_generation': 'GPU.S3'},
 {'action': 'dismiss', 'dismiss_age': 84, 'server_generation': 'GPU.S3'}]

In [3]:
def get_solution_from_action_comb(action_comb):
    solution = []
    for action in action_comb:
        if action["action"] == "buy":
            datacenter_id = action["datacenter_id"]
            server_generation = action["server_generation"]
            solution.append({"time_step": 1, 
                             "datacenter_id": datacenter_id, 
                             "server_generation": server_generation, 
                             "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", 
                             "action": "buy"})
        elif action["action"] == "dismiss":
            dismiss_age = action["dismiss_age"]
            solution.append({"time_step": 1+dismiss_age, 
                             "datacenter_id": datacenter_id, 
                             "server_generation": server_generation, 
                             "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", 
                             "action": "dismiss"})
        elif action["action"] == "move":
            datacenter_id = action["datacenter_id"]
            move_age = action["move_age"]
            solution.append({"time_step": 1+move_age, 
                             "datacenter_id": datacenter_id, 
                             "server_generation": server_generation, 
                             "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", 
                             "action": "move"})
    solution = pd.DataFrame(solution)
    return solution
solution = get_solution_from_action_comb(action_comb)


In [4]:
df_servers = pd.read_csv('./data/servers.csv')
df_datacenters = pd.read_csv('./data/datacenters.csv')


In [5]:
import re

df_results = []
# for action_string in list_action_strings:
for action_string in ["CPU.S1|buy-dismiss|DC1,0.6-24"]:
    action_comb = parse_action_string(action_string)
    solution = get_solution_from_action_comb(action_comb)
    server_generation = action_comb[0]["server_generation"]
    latency_sensitivity = df_datacenters[df_datacenters['datacenter_id'] == action_comb[0]["datacenter_id"]]['latency_sensitivity'].iloc[0]

    server_capacity = df_servers[df_servers['server_generation'] == server_generation]['capacity'].iloc[0]
    average_U = action_comb[0]["average_U"]
    demand = pd.DataFrame({
        "time_step": list(range(1,168+1)),
        "server_generation": [server_generation]*168,
        "high": [0]*168,
        "medium": [0]*168,
        "low": [0]*168
    })
    demand[latency_sensitivity] = server_capacity * average_U
    evaluation_nofail.FAILTURE_RATE = 0.07260491698699582 * 0.0
    score = evaluation_nofail.get_evaluation(solution, 
                                            demand,
                                            datacenters,
                                            servers,
                                            selling_prices,
                                            time_steps=196, 
                                            verbose=True,
                                            random_demand=False)
    real_score = evaluation_fail.get_evaluation(solution, 
                                            demand,
                                            datacenters,
                                            servers,
                                            selling_prices,
                                            time_steps=196, 
                                            verbose=False,
                                            random_demand=False)
    print(action_string, score, real_score)
    df_results.append([action_string, score, real_score])
    
df_results = pd.DataFrame(df_results, columns=['action_string', 'score', 'real_score'])
df_results

{'time-step': 1, 'O': np.float64(-93.76), 'U': np.float64(0.6), 'L': np.float64(0.0104), 'P': np.float64(-15001.0), 'R': np.float64(360.0), 'C': np.float64(15361.0)}
{'time-step': 2, 'O': np.float64(-93.54), 'U': np.float64(0.6), 'L': np.float64(0.0208), 'P': np.float64(17.0), 'R': np.float64(360.0), 'C': np.float64(343.0)}
{'time-step': 3, 'O': np.float64(-92.95), 'U': np.float64(0.6), 'L': np.float64(0.0312), 'P': np.float64(31.6), 'R': np.float64(360.0), 'C': np.float64(328.4)}
{'time-step': 4, 'O': np.float64(-91.85), 'U': np.float64(0.6), 'L': np.float64(0.0417), 'P': np.float64(44.0), 'R': np.float64(360.0), 'C': np.float64(316.0)}
{'time-step': 5, 'O': np.float64(-90.14), 'U': np.float64(0.6), 'L': np.float64(0.0521), 'P': np.float64(54.76), 'R': np.float64(360.0), 'C': np.float64(305.24)}
{'time-step': 6, 'O': np.float64(-87.73), 'U': np.float64(0.6), 'L': np.float64(0.0625), 'P': np.float64(64.21), 'R': np.float64(360.0), 'C': np.float64(295.79)}
{'time-step': 7, 'O': np.float

,action_string,score,real_score
0,"CPU.S1|buy-dismiss|DC1,0.6-24",111.001899,122.544012


# buy-dismiss (using formula)

In [6]:
from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function, get_known, get_evaluation
import pandas as pd
import numpy as np

demand, datacenters, servers, selling_prices = load_problem_data(path='./data_test')

df_servers = pd.read_csv('./data/servers.csv')
df_datacenters = pd.read_csv('./data/datacenters.csv')
df_selling_price = pd.read_csv('./data/selling_prices.csv')


In [7]:
list_action_strings = []
for server_generation in df_servers['server_generation'].values:
    for datacenter_id in df_datacenters['datacenter_id'].values:
        for average_U in np.linspace(0.0, 1.0, 21)[1:]:
            life_expectancy = df_servers[df_servers['server_generation'] == server_generation]['life_expectancy'].iloc[0]
            for dismiss_age in range(life_expectancy, 0, -12):
                action_string = f"{server_generation}|buy-dismiss|{datacenter_id},{np.round(average_U,2)}-{dismiss_age}"
                list_action_strings.append(action_string)
len(list_action_strings)

4480

In [8]:
df_results = []
# for action_string in ["CPU.S1|buy-dismiss|DC1,0.6-24"]:
for action_string in list_action_strings:
    action_comb = parse_action_string(action_string)

    server_generation = action_comb[0]["server_generation"]
    datacenter_id = action_comb[0]["datacenter_id"]

    df_cur_server = df_servers[df_servers['server_generation'] == server_generation]
    df_cur_datacenter = df_datacenters[df_datacenters['datacenter_id'] == datacenter_id]

    cost_of_energy = df_cur_datacenter['cost_of_energy'].iloc[0]
    latency_sensitivity = df_cur_datacenter['latency_sensitivity'].iloc[0]

    selling_price = df_selling_price[np.logical_and(df_selling_price['server_generation'] == server_generation, 
                                                    df_selling_price['latency_sensitivity'] == latency_sensitivity)]['selling_price'].iloc[0]


    life_expectancy = df_cur_server['life_expectancy'].iloc[0]
    purchase_price = df_cur_server['purchase_price'].iloc[0]
    energy_consumption_per_day = df_cur_server['energy_consumption'].iloc[0]
    energy_cost_per_day = energy_consumption_per_day * cost_of_energy
    average_mantainance_cost_per_day = df_cur_server['average_maintenance_fee'].iloc[0]
    server_capacity = df_cur_server['capacity'].iloc[0]
    dismiss_age = action_comb[1]["dismiss_age"]
    failure_rate = 0.07260491698699582
    failure_rate = 0.0
    average_U = action_comb[0]["average_U"]
    
    list_objective_per_day = []
    for i in range(1, dismiss_age+1):
        Z_fail = ((1-failure_rate) * server_capacity).astype(int)
        demand_met = np.minimum(Z_fail, average_U * server_capacity)
        U = demand_met / Z_fail
        L = i / life_expectancy
        R = demand_met * selling_price
        C = average_mantainance_cost_per_day * (1 + 1.5 * i / life_expectancy * np.log2(1.5 * i / life_expectancy)) + energy_cost_per_day
        if i == 1:
            C += purchase_price
        P = R - C
        list_objective_per_day.append([U, L, P])


    list_objective_per_day = np.asarray(list_objective_per_day)
    objective_per_day = list_objective_per_day[:,0]* list_objective_per_day[:,2]
    # objective_per_day = list_objective_per_day[:,0] * list_objective_per_day[:,1] * list_objective_per_day[:,2]
    total_objective = np.sum(objective_per_day)
    print(action_string, total_objective)
    df_results.append([action_string, total_objective])

CPU.S1|buy-dismiss|DC1,0.05-96 -2333.5178219457957
CPU.S1|buy-dismiss|DC1,0.05-84 -1996.323911425124
CPU.S1|buy-dismiss|DC1,0.05-72 -1718.783212901704
CPU.S1|buy-dismiss|DC1,0.05-60 -1493.7079585770327
CPU.S1|buy-dismiss|DC1,0.05-48 -1312.5932659255063
CPU.S1|buy-dismiss|DC1,0.05-36 -1165.0184767182795
CPU.S1|buy-dismiss|DC1,0.05-24 -1037.508365594626
CPU.S1|buy-dismiss|DC1,0.05-12 -910.8782305224411
CPU.S1|buy-dismiss|DC1,0.1-96 -4379.0356438915915
CPU.S1|buy-dismiss|DC1,0.1-84 -3740.647822850248
CPU.S1|buy-dismiss|DC1,0.1-72 -3221.566425803408
CPU.S1|buy-dismiss|DC1,0.1-60 -2807.4159171540655
CPU.S1|buy-dismiss|DC1,0.1-48 -2481.1865318510127
CPU.S1|buy-dismiss|DC1,0.1-36 -2222.036953436559
CPU.S1|buy-dismiss|DC1,0.1-24 -2003.016731189252
CPU.S1|buy-dismiss|DC1,0.1-12 -1785.7564610448821
CPU.S1|buy-dismiss|DC1,0.15-96 -6136.553465837387
CPU.S1|buy-dismiss|DC1,0.15-84 -5232.971734275371
CPU.S1|buy-dismiss|DC1,0.15-72 -4508.349638705111
CPU.S1|buy-dismiss|DC1,0.15-60 -3941.123875731098


In [9]:
df_results = pd.DataFrame(df_results, columns=['action_string', 'score'])
df_results['server_generation'] = df_results['action_string'].apply(lambda x: x.split("|")[0])
df_results = pd.DataFrame.join(df_results.set_index('server_generation'),
                  df_servers[['server_generation', 'slots_size']].set_index('server_generation'), 
                  on='server_generation', how='left').reset_index()
df_results['score_per_slot'] = df_results['score'] / df_results['slots_size']
df_results = df_results.sort_values(by=['score_per_slot', 'action_string'], ascending=False).reset_index(drop=True)
df_results.to_csv("df_single_server_results_v6_buydismiss_UP.csv", index=False)
df_results

,server_generation,action_string,score,slots_size,score_per_slot
0,GPU.S3,"GPU.S3|buy-dismiss|DC3,1.0-96",1.384690e+06,4,346172.449799
1,GPU.S3,"GPU.S3|buy-dismiss|DC4,1.0-96",1.344370e+06,4,336092.449799
2,GPU.S3,"GPU.S3|buy-dismiss|DC2,1.0-96",1.313650e+06,4,328412.449799
3,GPU.S3,"GPU.S3|buy-dismiss|DC3,1.0-84",1.221388e+06,4,305347.124181
4,GPU.S3,"GPU.S3|buy-dismiss|DC3,0.95-96",1.216959e+06,4,304239.827309
...,...,...,...,...,...
4475,GPU.S3,"GPU.S3|buy-dismiss|DC3,0.2-96",-5.483804e+04,4,-13709.510040
4476,GPU.S2,"GPU.S2|buy-dismiss|DC4,0.2-96",-5.645529e+04,4,-14113.821285
4477,GPU.S3,"GPU.S3|buy-dismiss|DC3,0.15-96",-5.668053e+04,4,-14170.132530
4478,GPU.S3,"GPU.S3|buy-dismiss|DC4,0.15-96",-6.272853e+04,4,-15682.132530


# buy-move-dismiss

### simulation

In [10]:
import re

df_results = []
for action_string in ["GPU.S3|buy-move-dismiss|DC3,0.5-DC2,0.5,86-96"]:
    action_comb = parse_action_string(action_string)
    solution = get_solution_from_action_comb(action_comb)
    server_generation = action_comb[0]["server_generation"]
    latency_sensitivity_1 = df_datacenters[df_datacenters['datacenter_id'] == action_comb[0]["datacenter_id"]]['latency_sensitivity'].iloc[0]
    latency_sensitivity_2 = df_datacenters[df_datacenters['datacenter_id'] == action_comb[1]["datacenter_id"]]['latency_sensitivity'].iloc[0]

    server_capacity = df_servers[df_servers['server_generation'] == server_generation]['capacity'].iloc[0]
    average_U = action_comb[0]["average_U"]
    demand = pd.DataFrame({
        "time_step": list(range(1,168+1)),
        "server_generation": [server_generation]*168,
        "high": [0]*168,
        "medium": [0]*168,
        "low": [0]*168
    })
    demand[latency_sensitivity_1] = server_capacity * average_U
    demand[latency_sensitivity_2] = server_capacity * average_U

    score = evaluation_nofail.get_evaluation(solution, 
                                            demand,
                                            datacenters,
                                            servers,
                                            selling_prices,
                                            time_steps=196, 
                                            verbose=False,
                                            random_demand=False)
    real_score = evaluation_fail.get_evaluation(solution, 
                                            demand,
                                            datacenters,
                                            servers,
                                            selling_prices,
                                            time_steps=196, 
                                            verbose=False,
                                            random_demand=False)

    print(action_string, score, real_score)
    df_results.append([action_string, score, real_score])
df_results = pd.DataFrame(df_results, columns=['action_string', 'score', 'real_score'])

GPU.S3|buy-move-dismiss|DC3,0.5-DC2,0.5,86-96 112586.80038495417 128670.6290113762


### formula

In [11]:
from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function, get_known, get_evaluation
import pandas as pd
import numpy as np

demand, datacenters, servers, selling_prices = load_problem_data(path='./data_test')

df_servers = pd.read_csv('./data/servers.csv')
df_datacenters = pd.read_csv('./data/datacenters.csv')
df_selling_price = pd.read_csv('./data/selling_prices.csv')


In [12]:
df_servers_dict = df_servers.set_index('server_generation').to_dict('index')
df_datacenters_dict = df_datacenters.set_index('datacenter_id').to_dict('index')
df_selling_price_dict = df_selling_price.set_index(['server_generation', 'latency_sensitivity']).to_dict('index')

In [13]:
df_selling_price_dict[('CPU.S1', 'medium')]

{'selling_price': 15.0}

In [14]:
('CPU.S1', 'medium')

('CPU.S1', 'medium')

In [15]:
list_action_strings = []
for server_generation in df_servers['server_generation'].values:
    life_expectancy = df_servers_dict[server_generation]['life_expectancy']

    for datacenter_id_1 in df_datacenters['datacenter_id'].values:
        for datacenter_id_2 in df_datacenters['datacenter_id'].values:
            if datacenter_id_1 == datacenter_id_2:
                continue
            for average_U_1 in np.linspace(0.0, 1.0, 21)[1:]:
                for average_U_2 in np.linspace(0.0, 1.0, 21)[1:]:

                    for dismiss_age in range(life_expectancy, 0, -12):
                        for move_age in range(life_expectancy, 0, -12):
                            if move_age >= dismiss_age:
                                continue
                            action_string = f"{server_generation}|buy-move-dismiss|{datacenter_id_1},{np.round(average_U_1,2)}-{datacenter_id_2},{np.round(average_U_2,2)},{move_age}-{dismiss_age}"
                            list_action_strings.append(action_string)
len(list_action_strings)

940800

In [16]:
import concurrent.futures
from tqdm import tqdm



df_results = []
for action_string in tqdm(list_action_strings):
    action_comb = parse_action_string(action_string)

    server_generation = action_comb[0]["server_generation"]
    datacenter_id_1 = action_comb[0]["datacenter_id"]
    datacenter_id_2 = action_comb[1]["datacenter_id"]

    average_U_1 = action_comb[0]["average_U"]
    average_U_2 = action_comb[1]["average_U"]

    df_cur_server = df_servers_dict[server_generation]
    df_cur_datacenter_1 = df_datacenters_dict[datacenter_id_1]
    df_cur_datacenter_2 = df_datacenters_dict[datacenter_id_2]

    cost_of_energy_1 = df_cur_datacenter_1['cost_of_energy']
    latency_sensitivity_1 = df_cur_datacenter_1['latency_sensitivity']

    cost_of_energy_2 = df_cur_datacenter_2['cost_of_energy']
    latency_sensitivity_2 = df_cur_datacenter_2['latency_sensitivity']


    selling_price_1 = df_selling_price_dict[(server_generation,latency_sensitivity_1)]['selling_price']
    selling_price_2 = df_selling_price_dict[(server_generation,latency_sensitivity_2)]['selling_price']

    energy_consumption_per_day = df_cur_server['energy_consumption']
    energy_cost_per_day_1 = energy_consumption_per_day * cost_of_energy_1
    energy_cost_per_day_2 = energy_consumption_per_day * cost_of_energy_2


    life_expectancy = df_cur_server['life_expectancy']
    purchase_price = df_cur_server['purchase_price']
    average_mantainance_cost_per_day = df_cur_server['average_maintenance_fee']
    server_capacity = df_cur_server['capacity']
    cost_of_moving = df_cur_server['cost_of_moving']

    move_age = action_comb[1]["move_age"]
    dismiss_age = action_comb[2]["dismiss_age"]
    failure_rate = 0.0
    # failure_rate = 0.07260491698699582
    
    list_objective_per_day = []
    for i in range(1, dismiss_age+1):
        selling_price = selling_price_1 if i < move_age + 1 else selling_price_2
        energy_cost_per_day = energy_cost_per_day_1 if i < move_age + 1 else energy_cost_per_day_2
        average_U = average_U_1 if i < move_age + 1 else average_U_2
        Z_fail = int((1-failure_rate) * server_capacity)
        demand_met = np.minimum(Z_fail, average_U * server_capacity)

        U = demand_met / Z_fail
        L = i / life_expectancy
        R = demand_met * selling_price
        C = average_mantainance_cost_per_day * (1 + 1.5 * i / life_expectancy * np.log2(1.5 * i / life_expectancy)) + energy_cost_per_day
        if i == 1:
            C += purchase_price
        if i == move_age + 1:
            C += cost_of_moving
        P = R - C
        # print(i, U, np.round(L,2), P)
        list_objective_per_day.append([U, L, P])


    list_objective_per_day = np.asarray(list_objective_per_day)
    objective_per_day = list_objective_per_day[:,0] * list_objective_per_day[:,2]
    total_objective = np.sum(objective_per_day)

    df_results.append([action_string, total_objective])
df_results = pd.DataFrame(df_results, columns=['action_string', 'score'])
df_results

100%|██████████| 940800/940800 [03:25<00:00, 4585.47it/s]


,action_string,score
0,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,84-96",-2398.517822
1,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,72-96",-2413.517822
2,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,60-96",-2428.517822
3,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,48-96",-2443.517822
4,"CPU.S1|buy-move-dismiss|DC1,0.05-DC2,0.05,36-96",-2458.517822
...,...,...
940795,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,24-48",650280.884788
940796,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,12-48",655320.884788
940797,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,24-36",446422.159146
940798,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,12-36",451462.159146


In [17]:
df_results = pd.DataFrame(df_results, columns=['action_string', 'score'])
df_results['server_generation'] = df_results['action_string'].apply(lambda x: x.split("|")[0])
df_results = pd.DataFrame.join(df_results.set_index('server_generation'),
                  df_servers[['server_generation', 'slots_size']].set_index('server_generation'), 
                  on='server_generation', how='left').reset_index()
df_results['score_per_slot'] = df_results['score'] / df_results['slots_size']
df_results = df_results.sort_values(by=['score_per_slot', 'action_string'], ascending=False).reset_index(drop=True)
df_results.to_csv("df_single_server_results_v6_buymovedismiss_UP.csv", index=False)
df_results

,server_generation,action_string,score,slots_size,score_per_slot
0,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,12-96",1.378650e+06,4,344662.449799
1,GPU.S3,"GPU.S3|buy-move-dismiss|DC3,1.0-DC4,1.0,84-96",1.378650e+06,4,344662.449799
2,GPU.S3,"GPU.S3|buy-move-dismiss|DC3,1.0-DC2,1.0,84-96",1.374810e+06,4,343702.449799
3,GPU.S3,"GPU.S3|buy-move-dismiss|DC2,1.0-DC3,1.0,12-96",1.374810e+06,4,343702.449799
4,GPU.S3,"GPU.S3|buy-move-dismiss|DC4,1.0-DC3,1.0,24-96",1.373610e+06,4,343402.449799
...,...,...,...,...,...
940795,GPU.S3,"GPU.S3|buy-move-dismiss|DC3,0.4-DC4,0.15,12-96",-8.102921e+04,4,-20257.303320
940796,GPU.S3,"GPU.S3|buy-move-dismiss|DC1,0.4-DC4,0.15,12-96",-8.141321e+04,4,-20353.303320
940797,GPU.S3,"GPU.S3|buy-move-dismiss|DC1,0.55-DC4,0.15,12-96",-8.170522e+04,4,-20426.305794
940798,GPU.S3,"GPU.S3|buy-move-dismiss|DC1,0.45-DC4,0.15,12-96",-8.254255e+04,4,-20635.637478
